**Recuerda que una vez abierto, Da clic en "Copiar en Drive", de lo contrario no podras almacenar tu progreso**

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [ ]:
#configuración del laboratorio
# Ejecuta esta celda!
%load_ext autoreload
%autoreload 2
in_colab = True
import os

if not in_colab:
    import sys ; sys.path.append('../commons/utils/');
else: 
    os.system('wget https://raw.githubusercontent.com/mariabda2/ML_2022/master/Labs/commons/utils/general.py -O general.py')
    from general import configure_lab5_1
    configure_lab5_1()

import tensorflow as tf
tf.get_logger().setLevel('INFO')
import seaborn as sns
from lab5 import *
GRADER = part_1()

# Laboratorio 5 - Parte 1. Redes recurrentes

En este laboratorio vamos a explorar la creación de redes recurrentes (RNN). De Igual manera vamos a explorar algunas funcionalidades de [la librería TensorFlow](https://www.tensorflow.org/). Recordemos que esta librería es uno de los estándares para entrenar redes neuronales e implementa varios de los conceptos y mejoras de Deep Learning. Este laboratorio sirve como un abrebocas para esta librería, ya que esta tiene muchas funcionalidades que no vamos a explorar en profundidad.
 
Las RNN son diseñadas para resolver problemas donde nuestros datos tienen un orden o una secuencia. Un tipo de estos problemas, son problemas de series de tiempo. Para este laboratorio vamos usar un [conjunto de datos de clima](https://www.bgc-jena.mpg.de/wetter/) el cual fue recolectado por el [instituto Max Planck](https://www.bgc-jena.mpg.de/). Este conjunto de datos contiene 14 características como la temperatura del aire, la presión atmosférica y humedad, los cuales fueron medidos cada 10 minutos desde el 2003. Para aumentar la eficiencia, solo usaremos muestras entre 2012 y 2016.
 
Este laboratorio usa algunas ideas presentadas [en este tutorial publicado en la página de tensorflow](https://www.tensorflow.org/tutorials/structured_data/time_series). La invitación es que puedas explorar este tutorial luego de desarrollar esta guía.

Vamos a descargar el conjunto de datos.

In [ ]:
# Referenciar el link con el procesamiento de datos.
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

Como se mencionó el conjunto de datos tiene muestras cada 10 minutos y 14 variables. En la siguiente celda cargaremos el conjunto de datos y aplicaremos dos restricciones que nos ayudarán a facilitar el problema.
 
1. Usaremos muestras de cada hora, en lugar de usar las muestras con la frecuencia de 10 minutos original.
2. Filtramos a partir del año 2012. 
3. Usaremos solo una variable: `T (degC)`.

In [ ]:
df = pd.read_csv(csv_path)
# Aplicamos el siguiente slice [start:stop:step]. 
# Significa tomaremos intervalos cada 6 pasos. 
#  Esto quiere decir tomar puntos cada hora
df = df[5::6]
# Transformamos la columna de fecha.
# Usamos las fechas como indice.
df.index =  pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')
onwards_2012 = (df.index.year >= 2012)
df = df.loc[onwards_2012, ['T (degC)']].copy()
df.head()

En la siguiente celda visualizamos los datos. Debemos observar el aparente periodo que existe en nuestra variable. ¿cada cuantos puntos parece repertirse el patrón de la serie?

In [ ]:
plot_cols = ['T (degC)']
plot_features = df[plot_cols]
_ = plot_features.plot(subplots=True, figsize = (15,6))

Hacemos un acercamiento en diferentes fechas, para ver los ciclos diarios:

In [ ]:
enero_2012 = (df.index.month == 1) & (df.index.year == 2012) 
plot_features = df.loc[enero_2012, plot_cols]
_ = plot_features.plot(subplots=True, figsize = (15,6))

In [ ]:
junio_2014 = (df.index.month == 6) & (df.index.year == 2014) 
plot_features = df.loc[junio_2014, plot_cols]
_ = plot_features.plot(subplots=True, figsize = (15,6))

En nuestro primer ejercicio vamos a explorar, el patrón que observamos en la grafica anterior.

La librería statsmodel [tiene una función que nos sirve para analizar esta relación](https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.plot_acf.html).


## Ejercicio 1 - Exploración del problema

Esta función realiza una operación cuyos detalles son explicados en mayor profundidad en [esta buena entrada de blog](https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/). En  nuestro laboratorio lo que no interesa entender:

1. El valor varia entre 1.0 y -1.0. 
2. Cuando el valor de la correlación es 1.0, corresponde el valor maximo indicando una relación positiva entre la variable y su correspondiente lag o retraso.
3. Cuando el valor de la correlación es -1.0, corresponde el valor mínimo indicando una relación negativa entre la variable y su correspondiente lag o retraso.
4. 0.0 indica que los valores no están relacionados.
5. El eje X indica el número de retrasos. Si el valor de la correlación en el lag  5 es igual 0.75, indica una relación positiva alta entre el quinto retraso anterior en la mayoria de muestras de nuestra variable objetivo.

Ahora, grafiquemos la auto-correlación. Observa las variables `dias_to_plot` y `horas_dia`.  Puedes variar los valores de estas variables y observar como cambia la grafica.

In [ ]:
from statsmodels.graphics import tsaplots

fig, ax = plt.subplots(figsize = (18,6))
dias_to_plot = 15
horas_dia = 24
TOTAL_MUESTRAS = dias_to_plot*horas_dia
rango = list(range(1,TOTAL_MUESTRAS, horas_dia))
# Muestra la aucorrelación de la serie de tiempo.
fig = tsaplots.plot_acf(df[plot_cols], lags=range(1, TOTAL_MUESTRAS), ax = ax)
# Grafica indicadores para observar el patros
ax.scatter(rango , 0.75*np.ones(dias_to_plot), c = 'r', marker = "v")
ax.set_xticks(rango,  [f'ciclo {int(c/horas_dia)}\n de 24 lags' for c in rango])
plt.xticks(rotation=45)
plt.show()

Reforzando el entendimiento, observando la gráfica anterior:
 
1. Los lags varían entre 0 y 24, es decir evaluamos que tan relacionado está el valor "presente" con las muestras de 1,2,3,..., hasta 24 horas , tenemos una autocorrelación que comienza alta ($\approx 1$) y va decreciendo hasta $\approx 0.75$.
2. Luego del lag 25, otro ciclo parece repetirse, pero los valores de autocorrelación son más bajos que en el primer ciclo.
3. Se observa un ciclo cada 24 puntos, pero por cada nuevo ciclo el valor de la autocorrelación disminuye, por ejemplo para el "ciclo 10" el valor máximo de autocorrelación ya está por debajo de 0.75. **PISTA**: ¿que tan relacionada está la temperatura de el dia de *hoy* con la temperatura de hace 10 días? ¿Esta relación es mayor o menor que la temperatura de hace 24 horas?

De acuerdo a la interpretación de la anterior grafica responde la siguiente pregunta:

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Cual podria ser el número minimo de muestras pasadas/retrasos que estan relacionados con el valor presente?
respuesta_1 = "" #@param {type:"string"}

Como sabemos de nuestra teoría, para poder aplicar RNN a una serie de tiempo debemos transformar nuestro datos en alguno de los siguientes problemas:
 
1. one to one
2. one to many
3. many to one
4. many to many
 
La siguiente función usa algunas de las funcionalidades de TensorFlow, para transformar los datos para transformar nuestro conjunto de datos en alguno de los anteriores tipo de problema. 

Explora un poco el codigo, pero en la siguientes celdas vamos a entender los efectos practicos de las funciones.

**NOTA**: Para este laboratorio, vamos enfocarnos más en el efecto practico de la función y no necesitas entender en profundidad las diferentes funcionalidades de la libreria.

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                train_df=None, 
                test_df=None,
                label_columns=plot_cols):
        self.train_df = train_df
        self.test_df = test_df

        self.label_columns = label_columns
        self.label_columns_indices = {name: i for i, name in
                                      enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                              enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total tamaño de la ventana: {self.total_window_size}',
            f'Inidice de entrada: {self.input_indices}',
            f'Indice a predicir: {self.label_indices}'])

def split_window(self, features=None, with_example= False):
    if with_example:
        # definir un Ejemplo.
        features = tf.stack([np.array(self.train_df[:self.total_window_size]),
                           np.array(self.train_df[100:100+self.total_window_size]),
                           np.array(self.train_df[200:200+self.total_window_size])])
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

    # configurar nuevamente el shape del vector.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    # asignar como ejemplos
    if with_example:
        self.example = inputs, labels
        
    return inputs, labels

def plot(self, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
      plt.subplot(max_n, 1, n+1)
      plt.ylabel(f'{plot_col}')
      plt.plot(self.input_indices, inputs[n, :, plot_col_index],
              label='Entradas', marker='.', zorder=-10)
      label_col_index = self.label_columns_indices.get(plot_col, None)
      plt.scatter(self.label_indices, labels[n, :, label_col_index],
                  edgecolors='k', label='Salida(s)', c='b', s=64)
      if n == 0:
        plt.legend()

# adicionar el metodo a clase.
WindowGenerator.split_window = split_window
# Adicionar el metodo a la clase
WindowGenerator.plot = plot


La siguiente celda codigo muestra el ejemplo de usar la función para generar muestras que usen seis puntos (ultimas seis horas), para predecir la siguiente hora.

![Imagen split window](https://www.tensorflow.org/static/tutorials/structured_data/images/split_window.png)


In [ ]:
w1 = WindowGenerator(train_df = df, input_width=6, label_width=1, shift=1)
w1

In [ ]:
w1.split_window(with_example=True)
w1.plot()

El anterior ejemplo transforma nuestro conjunto de datos en un problema `many-to-one`. Estamos usandos seis muestras, para predecir una muestra en el futuro. Ejecuta, analiza la siguiente celda y responde la pregunta.

In [ ]:
w1 = WindowGenerator(train_df = df, input_width=24, label_width=24, shift=24)
print(w1)
w1.split_window(with_example=True)
w1.plot()

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Analizando los parametros `input_width`, `label_width` y `shift` de `WindowGenerator`?
#@markdown ¿Cual de las opciones está mejor justificada?
 
#@markdown A) Usar `input_width=1, label_width=24, shift=24` representa un problema `one-to-one` y `input_width=24, label_width=24, shift=24` un problema `many-to-many`.
 
#@markdown B) Usar `input_width=1, label_width=24, shift=24` representa un problema `one-to-many` y `input_width=24, label_width=24, shift=24` un problema `many-to-many`.
 
#@markdown C) Usar `input_width=2, label_width=24, shift=24` representa un problema `one-to-many` y `input_width=24, label_width=24, shift=24` un problema `many-to-many`.
 
#@markdown D) Usar `input_width=1, label_width=24, shift=24` y `input_width=24, label_width=24, shift=24` representan un problema `many-to-many`.
 
#@markdown Selecciona dentro las lista desplegable
respuesta_2 = '' #@param ["", "A", "B", "C", "D"]

En Tensorflow, se debe crear de manera eficiente los conjuntos de datos, la siguiente celda, crea unos metodos que nos van ayudar a realizar este operación, usando la función que ya comprendimos anteriormente.

In [ ]:
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

WindowGenerator.train = train
WindowGenerator.test = test

Con el entendimiento de la función `WindowGenerator` vamos a realizar nuestro ejercicio de codigo, que consiste en forzar `WindowGenerator` para que siempre genere una "ventana" para un problema `many-to-one`. 
1. Lo unico que vamos a variar son las muestras pasadas/lags que usaremos para definir la ventana.
2. Dentro de la función se sugiere ya la partición entre el conjunto de entrenamiento y prueba. Observa lo siguiente:
    1. En este tipo de problemas una partición aleatoria no es una opción por que el orden es importante.
    2. Dentro de la función se usa una partición 80%-20%.

In [ ]:
#ejercicio de codigo
def many_to_one_custom(dataset, look_back=1):
    """Funcion que forza `WindowGenerator`
    
    dataset: matriz numpy con el conjunto de datos
    look_back: numero de retrasos con los cuales queremos construir
        las caracteristicas
    
    Retorna:
      un numpy array con los valores de X (debe ser una matrix)
      un numpy array con los valores de Y 
        (debe ser un vector columna, el # de renglones debe ser igual de renglones del numpy de X)

    """
    n = len(dataset)
    # Las primeras muestras hasta completar el 80%.
    train_df = dataset[0:int(n*0.8)]
    # Las siguientes muestras luego del 80% hasta llegar al final.
    test_df = dataset[int(n*0.8):]
    # Reemplazar por los valores correctos
    ventana =  WindowGenerator(train_df ... , 
                               test_df = ... ,
                               input_width=..., 
                               label_width=..., 
                               shift=1)   
    return ventana

In [ ]:
GRADER.run_test("ejercicio1", many_to_one_custom)

In [ ]:
# observemos el funcionamiento de nuestra funcion.
# Reemplaza los valores para observar el funcionamiento
window =  many_to_one_custom(df, 2)
print(window)
window.split_window(with_example=True)
window.plot()

## Ejercicio 2 - Experimentar con RNN

En el siguiente ejercicio vamos a crear una función para construir una RNN usando la libreria ya mencioanda.  

1. Asignar como funcion de perdida el valor del error medio absoluto.
2. Usar solo los objetos importados al inicio de la celda.

In [ ]:
# ejercicio de código 
# usar solo estos objetos
# importados
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.models import Sequential

def create_rnn_model(look_back, num_hidden_neurons):
    """funcion que crear modelo que usa mean_absolute_error
    como funcion de perdida
    RNN con base al número de lags y numero de neuronas

    parametros
      look_back (int): numero de retrasos a ejecutar
      num_hidden_neurons (int): numero neuronas en la capa oculta
    

    """
    # Se inicializa el modelo
    # Podemos asignar un nombre
    model = Sequential(name='rnn')
    # Adicionar una capa RNN
    # Reemplazar los valores
    # Asigna el nombre de rnn_layer
    rnn_layer = SimpleRNN(num_hidden_neurons, input_shape = (..., ...),  use_bias=True, name = ...) 
    # En tensorflow debemos adicionar la capa 
    # al modelo.
    model.add(rnn_layer)
    # La red termina con una capa Densa de una salida.
    model.add(Dense(1, name = "dense_layer"))
    # reemplazar la perdida por el parametro correcto.
    # dejar el optimizador `adam`
    model.build()
    model.compile(loss=..., optimizer='adam')
    return(model)

In [ ]:
GRADER.run_test("ejercicio2", create_rnn_model)

In [ ]:
rnn = create_rnn_model(look_back = 1,num_hidden_neurons = 2) 
rnn.summary()

In [ ]:
rnn = create_rnn_model(look_back = 2,num_hidden_neurons = 4) 
rnn.summary()

Con nuestra funcion que crea modelos, vamos experimentar variando los dos parametros:

- número de retrasos
- número de neuronas en la capa oculta

Otras condiciones: 
- Vamos a dejar fijo el # de epocas 25.
- Usaremos la metrica de error absoluto medio. Recordar que solo usamos la implementación de [sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).
- usar la función para crear la ventana.

In [ ]:
# Ejercicio de código
def experimentar_rnn(data, look_backs, hidden_neurons):
    """funcion que realiza experimentos para evaluar una RNN de elman usando
        el error absoluto medio como medida de error
    
    data: pd.Dataframe, dataset a usar
    look_back: List[int], lista con los numero de retrasos a evaluar
    hidden_neurons: List[int], list con el numero de neuronas en la capa oculta
    retorna:
        pd.Dataframe
    """
    # Normalizar usando min_max scaler.
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_df = data.copy()
    normalized_df[plot_cols] = scaler.fit_transform(data)
    resultados = pd.DataFrame()
    idx = 0
    for num_hidden_neurons in hidden_neurons:
        for look_back in look_backs:
            # aplicar la transformacion.
            window =  ... (normalized_df, look_back)
            model = create_rnn_model(look_back = look_back, num_hidden_neurons = num_hidden_neurons)
            MAX_EPOCHS =...
            model.fit(x = ... , epochs=MAX_EPOCHS,  verbose = 0)
            # predecimos en los conjuntos
            trainYPred  = model.predict(x=...)
            testYPred = model.predict(...)
            # se debe restringir el valor real.
            errorPrueba =  ...(y_true = window.test_df.head(testYPred.shape[0]), 
                                               y_pred = ...)
             # se debe restringir el valor real.
            errorEntrenamiento =  ...(y_true = window.train_df.head(trainYPred.shape[0]), 
                                                      y_pred= ... )
            resultados.loc[idx,'lags'] = ...
            resultados.loc[idx,'neuronas por capa'] = ...
            resultados.loc[idx,'Métrica rendimiento en entrenamiento'] = ...
            resultados.loc[idx,'Métrica de rendimiento prueba'] = ...
            idx+=1
            print("termina para", look_back, num_hidden_neurons)
    
    return (resultados)

In [ ]:
GRADER.run_test("ejercicio3", experimentar_rnn)

Ahora vamos a ver los resultados del experimentos:

1. variando los lags dejando las neuronas por capa fijas
2. variando las neuronas y dejando los retrasos fijos

experimente con diferentes configuraciones. Por la inicialización aleatorias los resultados pueden cambiar. Preste a los patrones que se van presentando y no a los valores exactos.

In [ ]:
# observa el comportamiento de los lags
resultadosRNN = experimentar_rnn(df, look_backs = [1,6,12,24], hidden_neurons=[15])
# plot
ax1  = sns.relplot(data = resultadosRNN, x = 'lags', y = 'Métrica de rendimiento prueba', kind = 'line', aspect = 2)
ax1.fig.suptitle('efecto del # retrasos')

In [ ]:
resultadosRNN = experimentar_rnn(df, look_backs = [24], hidden_neurons=[1,2,4,16])
ax2  = sns.relplot(data= resultadosRNN, x= 'neuronas por capa', y = 'Métrica de rendimiento prueba', kind = 'line', aspect = 2)
ax2.fig.suptitle('efecto del # neuronas')

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Por qué seguir aumentando los tiempos de retardo no implica siempre una mejora en la predicción del modelo?
respuesta_3 = "" #@param {type:"string"}

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Explique la principal diferencia entre un MLP y una red recurrente??
#@markdown ¿Cual de las opciones está mejor justificada?
 
#@markdown A) En un MLP no existe información compartida a través de las capas ocultas. En las RNN la capa de estado retro-alimenta las salidas previas ayudando a modelar problemas donde el orden en la secuencia es importante.
 
#@markdown B) En un MLP es posible mediante la capa oculta simula una capa de estao. En las RNN la capa de estado NO debe ser simula, esto ayuda a modelar problemas donde el orden en la secuencia es importante.
 
#@markdown C) El MLP y las RNN son equivalentes, solo que las RNN se entrenan usando Backpropagation through time (BPTT). Esto ayuda a modelar problemas donde el orden en la secuencia es importante.
 
#@markdown D) Las RNN se entrenan usando Backpropagation through time (BPTT), esto ayuda a modelar problemas donde el orden en la secuencia es importante. Un MLP no puede ser entrenado mediante BPTT.
 
#@markdown Selecciona dentro las lista desplegable
respuesta_4 = '' #@param ["", "A", "B", "C", "D"]

## Ejercicio 3 - Comparación con LSTM

En nuestro ultimo ejercicio, vamos a comparar los resultados obtenidos hasta ahora con una LSTM. Para ellos vamos a usar volver a usar [Tensorflow](https://www.tensorflow.org/?hl=es-419).



In [ ]:
#@title Pregunta Abierta
#@markdown ¿por qué una red LSTM puede ser más adecuada para resolver este problema? justifique
respuesta_5 = "" #@param {type:"string"}

Aca creamos el modelo LSTM usando tensorflow:

In [ ]:
from tensorflow.keras.layers import LSTM

def create_lstm_model(look_back, num_hidden_neurons):
    """funcion que crear modelo LSTM con base al número de lags y numero de neuronas"""
    model = Sequential(name='lstm')
    model.add(LSTM(num_hidden_neurons, input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.build()
    model.compile(loss='mean_squared_error', optimizer='adam')
    return(model)

Vamos aseguranos de completar el código para lograr:
-  Epocas = 50
-  Pasar los parametros el la función `create_lstm_model`
- Usaremos la metrica de error absoluto medio. Recordar que solo usamos la implementación de [sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).
- usar la función para crear la ventana.

In [ ]:
# Ejercicio de código
def experimentar_LSTM(data, look_backs, hidden_neurons):
    """funcion que realiza experimentos para evaluar LSTM usando
        el error absoluto medio como medida de error
    
    data: pd.Dataframe, dataset a usar
    look_back: List[int], lista con los numero de retrasos a evaluar
    hidden_neurons: List[int], list con el numero de neuronas en la capa oculta
    retorna: 
        pd.Dataframe
    """
    # Normalizar usando min_max scaler.
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_df = data.copy()
    normalized_df[plot_cols] = scaler.fit_transform(data)
    resultados = pd.DataFrame()
    idx = 0
    for num_hidden_neurons in hidden_neurons:
        for look_back in look_backs:
            # aplicar la transformacion.
            window =  ... (normalized_df, look_back)
            model = ... (look_back = look_back, num_hidden_neurons = num_hidden_neurons)
            # entrenemos el modelo.
            MAX_EPOCHS = 40
            model.fit(x = ... , epochs=MAX_EPOCHS,  verbose = 0)
            # predecimos en los conjuntos
            trainYPred  = model.predict(...)
            testYPred = model.predict(...)
            # se debe restringir el valor real.
            errorPrueba =  ...(y_true = window.test_df.head(testYPred.shape[0]), 
                                               y_pred=...)
             # se debe restringir el valor real.
            errorEntrenamiento =  ...(y_true = window.train_df.head(trainYPred.shape[0]), 
                                                      y_pred= ...)
            resultados.loc[idx,'lags'] = ...
            resultados.loc[idx,'neuronas por capa'] = ...
            resultados.loc[idx,'Métrica rendimiento en entrenamiento'] = ...
            resultados.loc[idx,'Métrica de rendimiento prueba'] = ...
            idx+=1
            print("termina para", look_back, num_hidden_neurons)
    
    return (resultados)

In [ ]:
# ignorar los prints!
GRADER.run_test("ejercicio4", experimentar_LSTM)

In [ ]:
# demora algunos minutos!
resultadosLSTM = experimentar_LSTM(df, [1,6,12,24], hidden_neurons=[4,8,16])

In [ ]:
# para ver los resultados
# en esta instruccion se va resaltar el mejor
# error y tiempo de entrenamiento
resultadosLSTM.style.highlight_min(color = 'green', axis = 0, subset = ['Métrica de rendimiento prueba'])

In [ ]:
GRADER.check_tests()

In [ ]:
#@title Integrantes
codigo_integrante_1 ='' #@param {type:"string"}
codigo_integrante_2 = ''  #@param {type:"string"}

----
esta linea de codigo va fallar, es de uso exclusivo de los profesores


In [ ]:
GRADER.grade()